## Minh Hoang Nguyen - e2304951

# Protocol 1: The Basic Key Transport Protocol

## Part 1: RSA and AES

### 1. RSA Private and Public Key Generation

In [32]:
import random, sys, os, math

def gcd(a, b):
    # Return the Greatest Common Divisor of a and b using Euclid's Algorithm
    while a != 0:
        a, b = b % a, a        
    return b

def findModInverse(a, m):
    # Return the modular inverse of a % m, which is
    # the number x such that a*x % m = 1

    if gcd(a, m) != 1:
        return None # No mod inverse exists if a & m aren't relatively prime.

    # Calculate using the Extended Euclidean Algorithm:
    u1, u2, u3 = 1, 0, a
    v1, v2, v3 = 0, 1, m
    while v3 != 0:
        q = u3 // v3 # Note that // is the integer division operator
        v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
    return u1 % m

def primeSieve(sieveSize):
    # Returns a list of prime numbers calculated using
    # the Sieve of Eratosthenes algorithm.

    sieve = [True] * sieveSize
    sieve[0] = False # Zero and one are not prime numbers.
    sieve[1] = False
    
    # Create the sieve:
    for i in range(2, int(math.sqrt(sieveSize)) + 1):
        pointer = i * 2
        while pointer < sieveSize:
            sieve[pointer] = False
            pointer += i

    # Compile the list of primes:
    primes = []
    for i in range(sieveSize):
        if sieve[i] == True:
            primes.append(i)

    return primes

def rabinMiller(num):
    # Returns True if num is a prime number.
    if num % 2 == 0 or num < 2:
        return False # Rabin-Miller doesn't work on even integers.
    if num == 3:
        return True
    s = num - 1
    t = 0
    while s % 2 == 0:
        # Keep halving s until it is odd (and use t
        # to count how many times we halve s):
        s = s // 2
        t += 1
    for trials in range(5): # Try to falsify num's primality 5 times.
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # (This test does not apply if v is 1.)
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True

LOW_PRIMES = primeSieve(100)

def isPrime(num):
    # Return True if num is a prime number. This function does a quicker
    # prime number check before calling rabinMiller().
    if (num < 2):
        return False # 0, 1, and negative numbers are not prime.
    # See if any of the low prime numbers can divide num:
    for prime in LOW_PRIMES:
        if (num % prime == 0):
            return False
        if (num == prime):
            return True
    # If all else fails, call rabinMiller() to determine if num is a prime:
    return rabinMiller(num)

def generateLargePrime(keysize=1024):
    # Return a random prime number that is keysize bits in size.
    i=0
    while True:
        num = random.randrange(2**(keysize-1), 2**(keysize))
        #print('random number', i, hex(num))
        i = i + 1
        if isPrime(num):
            #print('number of tries:', i)
            return num

def makeKeyFiles(dir, name, publicKey, privateKey):
    # Creates two files 'x_pubkey.txt' and 'x_privkey.txt' (where x
    # is the value in name) with the n,e and d,e integers written in
    # them, delimited by a comma.

    # Our safety check will prevent us from overwriting our old key files:
    isExist = os.path.exists(dir)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(dir)
    if os.path.exists('%s/%s_pubkey.txt' % (dir, name)) or os.path.exists('%s/%s_privkey.txt' % (dir, name)):
        sys.exit('WARNING: The file %s_pubkey.txt or %s_privkey.txt already exists! Use a different name or delete these files and re-run this program.' % (dir, name))

    print()
    print('Writing public key to file %s_pubkey.txt...' % (name))
    fo = open('%s/%s_pubkey.txt' % (dir, name), 'w')
    fo.write(publicKey)
    fo.close()

    print()
    print('Writing private key to file %s_privkey.txt...' % (name))
    fo = open('%s/%s_privkey.txt' % (dir, name), 'w')
    fo.write(privateKey)
    fo.close()

def generateKey(dir, name,keySize):
    # Creates a public/private keys keySize bits in size.
    p = 0
    q = 0
    # Step 1: Create two prime numbers, p and q. Calculate n = p * q.
    print('Generating p & q primes...')
    while p == q:
        p = generateLargePrime(keySize)
        q = generateLargePrime(keySize)
    n = p * q

    # Step 2: Create a number e that is relatively prime to (p-1)*(q-1):
    print('Generating e that is relatively prime to (p-1)*(q-1)...')
    while True:
        # Keep trying random numbers for e until one is valid:
        e = random.randrange(2 ** (keySize - 1), 2 ** (keySize))
        if gcd(e, (p - 1) * (q - 1)) == 1:
            break

    # Step 3: Calculate d, the mod inverse of e:
    print('Calculating d that is mod inverse of e...')
    d = findModInverse(e, (p - 1) * (q - 1))

    publicKey = (n, e)
    privateKey = (n, d)

    final_publicKey = '%s,%s,%s' % (keySize, publicKey[0], publicKey[1])
    final_privateKey = '%s,%s,%s' % (keySize, privateKey[0], privateKey[1])

    makeKeyFiles(dir, name, final_publicKey, final_privateKey)

    return (final_publicKey, final_privateKey)

### 2. RSA Encryption

In [33]:
#RSA Encryption

DEFAULT_BLOCK_SIZE = 128 # 128 bytes
BYTE_SIZE = 256 # One byte has 256 different values.

def getBlocksFromText(message, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts a string message to a list of block integers. Each integer
    # represents 128 (or whatever blockSize is set to) string characters.

    messageBytes = message.encode('ascii') # convert the string to bytes

    blockInts = []
    for blockStart in range(0, len(messageBytes), blockSize):
        # Calculate the block integer for this block of text
        blockInt = 0
        for i in range(blockStart, min(blockStart + blockSize, len(messageBytes))):
            blockInt += messageBytes[i] * (BYTE_SIZE ** (i % blockSize))
        blockInts.append(blockInt)
    return blockInts

def readKeyFile(keyFilename):
    # Given the filename of a file that contains a public or private key,
    # return the key as a (n,e) or (n,d) tuple value.
    fo = open(keyFilename)
    content = fo.read()
    fo.close()
    keySize, n, EorD = content.split(',')
    return (int(keySize), int(n), int(EorD))

def encryptMessage(message, key, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts the message string into a list of block integers, and then
    # encrypts each block integer. Pass the PUBLIC key to encrypt.
    encryptedBlocks = []
    n, e = key

    for block in getBlocksFromText(message, blockSize):
        # ciphertext = plaintext ^ e mod n
        encryptedBlocks.append(pow(block, e, n))
    return encryptedBlocks

def RSAEncryption(keyFilename, message, blockSize=DEFAULT_BLOCK_SIZE):
    # Using a key from a key file, encrypt the message and save it to a file. Returns the encrypted message string.
    keySize, n, e = readKeyFile(keyFilename)

    # Check that key size is greater than block size.
    if keySize < blockSize * 8: # * 8 to convert bytes to bits
        sys.exit('ERROR: Block size is %s bits and key size is %s bits. The RSA cipher requires the block size to be equal to or greater than the key size. Either decrease the block size or use different keys.' % (blockSize * 8, keySize))

    # Encrypt the message
    encryptedBlocks = encryptMessage(message, (n, e), blockSize)

    # Convert the large int values to one string value.
    for i in range(len(encryptedBlocks)):
        encryptedBlocks[i] = str(encryptedBlocks[i])
    encryptedContent = ','.join(encryptedBlocks)

    encryptedContent = '%s_%s_%s' % (len(message), blockSize, encryptedContent)
    return encryptedContent

### 3. RSA Decryption

In [34]:
# RSA Decryption

DEFAULT_BLOCK_SIZE = 128 # 128 bytes
BYTE_SIZE = 256 # One byte has 256 different values.

def getTextFromBlocks(blockInts, messageLength, blockSize=DEFAULT_BLOCK_SIZE):
    # Converts a list of block integers to the original message string.
    # The original message length is needed to properly convert the last
    # block integer.
    message = []
    for blockInt in blockInts:
        blockMessage = []
        for i in range(blockSize - 1, -1, -1):
            if len(message) + i < messageLength:
                # Decode the message string for the 128 (or whatever
                # blockSize is set to) characters from this block integer.
                asciiNumber = blockInt // (BYTE_SIZE ** i)
                blockInt = blockInt % (BYTE_SIZE ** i)
                blockMessage.insert(0, chr(asciiNumber))
        message.extend(blockMessage)
    return ''.join(message)

def decryptMessage(encryptedBlocks, messageLength, key, blockSize=DEFAULT_BLOCK_SIZE):
    # Decrypts a list of encrypted block ints into the original message
    # string. The original message length is required to properly decrypt
    # the last block. Be sure to pass the PRIVATE key to decrypt.
    decryptedBlocks = []
    n, d = key
    for block in encryptedBlocks:
        # plaintext = ciphertext ^ d mod n
        decryptedBlocks.append(pow(block, d, n))
    return getTextFromBlocks(decryptedBlocks, messageLength, blockSize)

def RSADecryption(dirfileName, keyFilename):
    # Using a key from a key file, encrypt the message and save it to a
    # file. Returns the encrypted message string.
    keySize, n, d = readKeyFile(keyFilename)
    encryptedText = readFile(dirfileName)

    # Read in the message length and the encrypted message from the file.
    messageLength, blockSize, encryptedMessage = encryptedText.split('_')
    messageLength = int(messageLength)
    blockSize = int(blockSize)

    # Check that key size is greater than block size.
    if keySize < blockSize * 8: # * 8 to convert bytes to bits
        sys.exit('ERROR: Block size is %s bits and key size is %s bits. The RSA cipher requires the block size to be equal to or greater than the key size. Did you specify the correct key file and encrypted file?' % (blockSize * 8, keySize))

    # Convert the encrypted message into large int values.
    encryptedBlocks = []
    for block in encryptedMessage.split(','):
        encryptedBlocks.append(int(block))

    # Decrypt the large int values.
    return decryptMessage(encryptedBlocks, messageLength, (n, d), blockSize)


### 4. AES Secret Symmetric Key Generation and makeFile with readFile functions

In [35]:
def generate_secret_key_for_AES_cipher():
    AES_key_length = 16
    secret_key = os.urandom(AES_key_length)
    byte_secret_key = bytearray(secret_key)
    hex_secret_symmetric_key = byte_secret_key.hex()
    hex_secret_symmetric_key = (f'0x{hex_secret_symmetric_key}')
    return hex_secret_symmetric_key

def makeFile(dir, fileName, content):
    # Our safety check will prevent us from overwriting our old key files:
    isExist = os.path.exists(dir)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(dir)
    if os.path.exists('%s/%s' % (dir, fileName)):
        sys.exit('WARNING: The file %s/%s already exists! Use a different name or delete these files and re-run this program.' % (dir, fileName))

    print()
    print('Creating %s/%s...' % (dir, fileName))
    fo = open('%s/%s' % (dir, fileName), 'w')
    fo.write(content)
    fo.close()

def readFile(dirfileName):
    fo = open(dirfileName)
    content = fo.read()
    fo.close()
    return content

### 5. Execution - Part 1

In [36]:
# Detail explaination in the submitted report
file_path_list = ['protocol-1-part-1-bob/bob_pubkey.txt','protocol-1-part-1-bob/bob_privkey.txt','protocol-1-part-1-alice/symkey.txt','protocol-1-part-1-alice/encryptedsymkey.txt','protocol-1-part-1-bob/decryptedsymkey.txt']
for file_path in file_path_list:
    if os.path.exists(file_path):
        os.remove(file_path)

#Bob
publicKey, privateKey = generateKey('protocol-1-part-1-bob','bob',2048)
print("\n------------------")
print('Bob Public Key: ', publicKey)
print("------------------")
print('Bob Private Key: ', privateKey)
print("------------------\n")

#Alice
hex_secret_symmetric_key = generate_secret_key_for_AES_cipher()
print(f"Secret Symmetric Key: {hex_secret_symmetric_key}")
makeFile('protocol-1-part-1-alice','symkey.txt',hex_secret_symmetric_key)
print("\n------------------")
encryptedSymKey = RSAEncryption('protocol-1-part-1-bob/bob_pubkey.txt', hex_secret_symmetric_key)
makeFile('protocol-1-part-1-alice','encryptedsymkey.txt',encryptedSymKey)
print('Encrypted Symmetric Key:')
print(encryptedSymKey)

#Bob
decryptedSymKey = RSADecryption('protocol-1-part-1-alice/encryptedsymkey.txt', 'protocol-1-part-1-bob/bob_privkey.txt')
makeFile('protocol-1-part-1-bob','decryptedsymkey.txt',decryptedSymKey)
print('Decrypted Symmetric Key:')
print(decryptedSymKey)

Generating p & q primes...
Generating e that is relatively prime to (p-1)*(q-1)...
Calculating d that is mod inverse of e...

Writing public key to file bob_pubkey.txt...

Writing private key to file bob_privkey.txt...

------------------
Bob Public Key:  2048,6271593624188173478408524379573971736578546204603227660660124953153481210582269934760858306943198585952469014952431001623919004225635870898827268869860354773772210341933960504039936959139368411062450444829702211435239302348346943501696013371524565273894273241582238395715807719570235995267909104977028170222389022291743396995634601293575766792205515175660322898184736303366652859721074982558457983136100311147391840242497584715065194687260949629118039707109191718878620541978362239178243873929988166140778049321239234541985192674114463081764230479752223396306046445263050071613955128798253108188418434705392858581457149211241052222434214678042055282885691593524154489250509210040531653825551423887056099567982974138706843405960801227504268

### 6. Round-key generation

In [37]:
Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)


def text2matrix(text):
    matrix = []
    for i in range(16):
        byte = (text >> (8 * (15 - i))) & 0xFF
        if i % 4 == 0:
            matrix.append([byte])
        else:
            matrix[int(i / 4)].append(byte)
    return matrix

def change_key(master_key):
    round_keys = text2matrix(master_key)
    print ('\n original key : \n', round_keys)
    for i in range(4, 4 * 11):
        round_keys.append([])
        if i % 4 == 0:
            byte = round_keys[i - 4][0]        \
                     ^ Sbox[round_keys[i - 1][1]]  \
                     ^ Rcon[int(i / 4)]
            round_keys[i].append(byte)

            for j in range(1, 4):
                    byte = round_keys[i - 4][j]    \
                         ^ Sbox[round_keys[i - 1][(j + 1) % 4]]
                    round_keys[i].append(byte)
        else:
            for j in range(4):
                    byte = round_keys[i - 4][j]    \
                         ^ round_keys[i - 1][j]
                    round_keys[i].append(byte)
    
    print ('\n cipher key : \n', round_keys)
    return(round_keys)      

### 7. AES Encryption

In [38]:
# learnt from http://cs.ucsb.edu/~koc/cs178/projects/JT/aes.c
xtime = lambda a: (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)


def matrix2text(matrix):
    text = 0
    for i in range(4):
        for j in range(4):
            text |= (matrix[i][j] << (120 - 8 * (4 * i + j)))
    return text

def block_encrypt(plaintext,round_keys):
        plain_state = text2matrix(plaintext)

        add_round_key(plain_state, round_keys[:4])

        for i in range(1, 10):
            round_encrypt(plain_state, round_keys[4 * i : 4 * (i + 1)])

        sub_bytes(plain_state)
        shift_rows(plain_state)
        add_round_key(plain_state, round_keys[40:])

        return matrix2text(plain_state)


def add_round_key(s, k):
        for i in range(4):
            for j in range(4):
                s[i][j] ^= k[i][j]


def round_encrypt(state_matrix, key_matrix):
    """Applies all four steps of one round of AES encryption."""
    sub_bytes(state_matrix)
    shift_rows(state_matrix)
    mix_columns(state_matrix)
    add_round_key(state_matrix, key_matrix)

def sub_bytes(s):
    """Substitute each byte in the state matrix using the Sbox."""
    for i in range(4):
        for j in range(4):
            s[i][j] = Sbox[s[i][j]]

def shift_rows(s):
    """Shifts the rows of the state matrix."""
    # print(s)
    # for r in range(1,len(s)):
    #     s[r] = s[r][r:] + s[r][:r]
    s[0][1], s[1][1], s[2][1], s[3][1] = s[1][1], s[2][1], s[3][1], s[0][1]
    
    # Third row is shifted left by two positions
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    
    # Fourth row is shifted left by three positions
    s[0][3], s[1][3], s[2][3], s[3][3] = s[3][3], s[0][3], s[1][3], s[2][3]

def mix_single_column(a):
        # please see Sec 4.1.2 in The Design of Rijndael
        t = a[0] ^ a[1] ^ a[2] ^ a[3]
        u = a[0]
        a[0] ^= t ^ xtime(a[0] ^ a[1])
        a[1] ^= t ^ xtime(a[1] ^ a[2])
        a[2] ^= t ^ xtime(a[2] ^ a[3])
        a[3] ^= t ^ xtime(a[3] ^ u)


def mix_columns(s):
        for i in range(4):
            mix_single_column(s[i])     

def encrypt(SymmetricKey,plaintext_message):
    encrypt_round_keys=change_key(int(SymmetricKey,16))
    print('\n plain-text message is: ', plaintext_message)
    hex_message = '0x' + plaintext_message.encode("utf-8").hex()
    
    if len(hex_message[2:]) > 32:
        hex_message = hex_message[2:]
        hex_parts = [hex_message[i:i+32] for i in range(0, len(hex_message), 32)]
        for hex_part in hex_parts:
            hex_part = '0x' + hex_part
            ciphertext_message = block_encrypt(int(hex_part, 16),encrypt_round_keys)
            isExist = os.path.exists('protocol-1-part-1-alice/ciphertext_message.txt')
            if not isExist:
                makeFile('protocol-1-part-1-alice', 'ciphertext_message.txt', str(ciphertext_message))
            else:
                with open('protocol-1-part-1-alice/ciphertext_message.txt', 'a') as file:
                    file.write(f"\n{ciphertext_message}")
    else:
        ciphertext_message = block_encrypt(int(hex_message, 16))
        print('\n cipher-text message is:',hex(ciphertext_message))
        makeFile('protocol-1-part-1-alice', 'ciphertext_message.txt', str(ciphertext_message))


### 8. AES Decryption

In [39]:
InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)


def block_decrypt(ciphertext, round_keys):
        cipher_state = text2matrix(ciphertext)

        add_round_key(cipher_state, round_keys[40:])
        inv_shift_rows(cipher_state)
        inv_sub_bytes(cipher_state)

        for i in range(9, 0, -1):
            round_decrypt(cipher_state, round_keys[4 * i : 4 * (i + 1)])

        add_round_key(cipher_state, round_keys[:4])

        return matrix2text(cipher_state)


def round_decrypt(state_matrix, key_matrix):
    "call to the four round operations"
    add_round_key(state_matrix, key_matrix)
    inv_mix_columns(state_matrix)
    inv_shift_rows(state_matrix)
    inv_sub_bytes(state_matrix)       

def inv_sub_bytes(s):
    "build the inv_sub_bytes transformation"
    for i in range(4):
        for j in range(4):
            s[i][j] = InvSbox[s[i][j]]

def inv_shift_rows(s):
    "build the inv_shift_rows transformation"
    s[0][1], s[1][1], s[2][1], s[3][1] = s[3][1], s[0][1], s[1][1], s[2][1]
    
    # Third row is shifted left by two positions
    s[0][2], s[1][2], s[2][2], s[3][2] = s[2][2], s[3][2], s[0][2], s[1][2]
    
    # Fourth row is shifted left by three positions
    s[0][3], s[1][3], s[2][3], s[3][3] = s[1][3], s[2][3], s[3][3], s[0][3]

def inv_mix_columns(s):
        # see Sec 4.1.3 in The Design of Rijndael
        for i in range(4):
            u = xtime(xtime(s[i][0] ^ s[i][2]))
            v = xtime(xtime(s[i][1] ^ s[i][3]))
            s[i][0] ^= u
            s[i][1] ^= v
            s[i][2] ^= u
            s[i][3] ^= v

        mix_columns(s)

def decrypt(ciphertext_message_path,decryptedSymmetricKey):
    plaintext_message = ''
    decrypt_round_keys=change_key(int(decryptedSymmetricKey,16))
    with open(ciphertext_message_path, 'r') as file:
        for hex_part in file:
            encrypted_ciphertext_message = str('0x' + hex(int(hex_part))[2:])
            decrypted = block_decrypt(int(encrypted_ciphertext_message, 16),decrypt_round_keys)
            byte_data = bytes.fromhex(hex(decrypted)[2:])
            plaintext_message += byte_data.decode("utf-8")

    return plaintext_message

In [40]:
file_path = 'protocol-1-part-1-alice/plaintext-message.txt'
if os.path.exists(file_path):
    os.remove(file_path)

#Alice
plaintext = "No pain no gain. No guts no glory. Cyberus is the BEST.xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
makeFile('protocol-1-part-1-alice','plaintext-message.txt',plaintext)


Creating protocol-1-part-1-alice/plaintext-message.txt...


### 10. Execution - Part 2

In [41]:
# Detail Explanation in the submitted report
#Alice
file_path = 'protocol-1-part-1-alice/ciphertext_message.txt'
if os.path.exists(file_path):
    os.remove(file_path)

SymmetricKey = readFile('protocol-1-part-1-alice/symkey.txt')
plaintext_message = readFile('protocol-1-part-1-alice/plaintext-message.txt')

encrypt(SymmetricKey, plaintext_message)


 original key : 
 [[129, 105, 7, 239], [144, 91, 202, 144], [112, 73, 27, 162], [129, 62, 98, 46]]

 cipher key : 
 [[129, 105, 7, 239], [144, 91, 202, 144], [112, 73, 27, 162], [129, 62, 98, 46], [50, 195, 54, 227], [162, 152, 252, 115], [210, 209, 231, 209], [83, 239, 133, 255], [239, 84, 32, 14], [77, 204, 220, 125], [159, 29, 59, 172], [204, 242, 190, 83], [98, 250, 205, 69], [47, 54, 17, 56], [176, 43, 42, 148], [124, 217, 148, 199], [95, 216, 11, 85], [112, 238, 26, 109], [192, 197, 48, 249], [188, 28, 164, 62], [211, 145, 185, 48], [163, 127, 163, 93], [99, 186, 147, 164], [223, 166, 55, 154], [215, 11, 1, 174], [116, 116, 162, 243], [23, 206, 49, 87], [200, 104, 6, 205], [210, 100, 188, 70], [166, 16, 30, 181], [177, 222, 47, 226], [121, 182, 41, 47], [28, 193, 169, 240], [186, 209, 183, 69], [11, 15, 152, 167], [114, 185, 177, 136], [81, 9, 109, 176], [235, 216, 218, 245], [224, 215, 66, 82], [146, 110, 243, 218], [248, 4, 58, 255], [19, 220, 224, 10], [243, 11, 162, 88], [97

In [42]:
#Bob
decryptedSymmetricKey = readFile('protocol-1-part-1-bob/decryptedsymkey.txt')
ciphertext_message_path = 'protocol-1-part-1-alice/ciphertext_message.txt'
plaintext_message = decrypt(ciphertext_message_path,decryptedSymmetricKey)
print()
print("Decrypted Plain-text Message: ", plaintext_message)


 original key : 
 [[129, 105, 7, 239], [144, 91, 202, 144], [112, 73, 27, 162], [129, 62, 98, 46]]

 cipher key : 
 [[129, 105, 7, 239], [144, 91, 202, 144], [112, 73, 27, 162], [129, 62, 98, 46], [50, 195, 54, 227], [162, 152, 252, 115], [210, 209, 231, 209], [83, 239, 133, 255], [239, 84, 32, 14], [77, 204, 220, 125], [159, 29, 59, 172], [204, 242, 190, 83], [98, 250, 205, 69], [47, 54, 17, 56], [176, 43, 42, 148], [124, 217, 148, 199], [95, 216, 11, 85], [112, 238, 26, 109], [192, 197, 48, 249], [188, 28, 164, 62], [211, 145, 185, 48], [163, 127, 163, 93], [99, 186, 147, 164], [223, 166, 55, 154], [215, 11, 1, 174], [116, 116, 162, 243], [23, 206, 49, 87], [200, 104, 6, 205], [210, 100, 188, 70], [166, 16, 30, 181], [177, 222, 47, 226], [121, 182, 41, 47], [28, 193, 169, 240], [186, 209, 183, 69], [11, 15, 152, 167], [114, 185, 177, 136], [81, 9, 109, 176], [235, 216, 218, 245], [224, 215, 66, 82], [146, 110, 243, 218], [248, 4, 58, 255], [19, 220, 224, 10], [243, 11, 162, 88], [97

## Part 2: Using Library

In [43]:
def makeFile(dir, fileName, content):
    # Our safety check will prevent us from overwriting our old key files:
    isExist = os.path.exists(dir)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(dir)
    if os.path.exists('%s/%s' % (dir, fileName)):
        sys.exit('WARNING: The file %s/%s already exists! Use a different name or delete these files and re-run this program.' % (dir, fileName))

    print()
    print('Creating %s/%s...' % (dir, fileName))
    fo = open('%s/%s' % (dir, fileName), 'w')
    fo.write(content)
    fo.close()

def readFile(dirfileName):
    fo = open(dirfileName)
    content = fo.read()
    fo.close()
    return content

In [44]:
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import os, sys

def generate_RSA_key_pair():
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

def RSA_encrypt(public_key, message):
    rsa_key = RSA.import_key(public_key)
    cipher = PKCS1_OAEP.new(rsa_key)
    return cipher.encrypt(message)

def RSA_decrypt(private_key, ciphertext):
    rsa_key = RSA.import_key(private_key)
    cipher = PKCS1_OAEP.new(rsa_key)
    return cipher.decrypt(ciphertext)

def AES_encrypt(key, plaintext):
    iv = get_random_bytes(16)
    cipher = AES.new(key, AES.MODE_OFB, iv)
    ct_bytes = cipher.encrypt(pad(plaintext, AES.block_size))  # Adding padding
    return iv + ct_bytes

def AES_decrypt(key, ciphertext):
    iv = ciphertext[:16]
    ct = ciphertext[16:]
    cipher = AES.new(key, AES.MODE_OFB, iv)
    return unpad(cipher.decrypt(ct), AES.block_size)  # Removing padding

In [45]:
file_path_list = ['protocol-1-part-2-bob/bob_privkey.txt','protocol-1-part-2-bob/bob_pubkey.txt','protocol-1-part-2-bob/decrypted_symkey.txt',
                 'protocol-1-part-2-alice/encryptedsymkey.txt','protocol-1-part-2-alice/ciphertext_message.txt','protocol-1-part-2-alice/symkey.txt']
for file_path in file_path_list:
    if os.path.exists(file_path):
        os.remove(file_path)

# Bob generates RSA keys and sends bob_pubkey.txt to Alice
private_key, public_key = generate_RSA_key_pair()
makeFile('protocol-1-part-2-bob', 'bob_privkey.txt', str(private_key)[2:-1])
makeFile('protocol-1-part-2-bob', 'bob_pubkey.txt', str(public_key)[2:-1])

# Alice generates random AES secret symmetric key and encrypts with Bob public_key
K = get_random_bytes(16)  # 128-bit AES key
makeFile('protocol-1-part-2-alice','symkey.txt', str(K)[2:-1])
bob_pubkey = readFile('protocol-1-part-2-bob/bob_pubkey.txt') #Alice reads Bob public key in file
bob_pubkey = bytes(bob_pubkey,'utf-8').replace(b'\\n', b'\n')
y1 = RSA_encrypt(bob_pubkey, K)
makeFile('protocol-1-part-2-alice','encryptedsymkey.txt', str(y1)[2:-1])
# Alice sends encryptedsymkey.txt to Bob

# Bob decrypts encryptedsymkey.txt with his private key to get secret symmetric key
alice_encryptedsymkey = readFile('protocol-1-part-2-alice/encryptedsymkey.txt') # Bob reads Alice encrypted symmetric key in file
alice_encryptedsymkey = bytes(alice_encryptedsymkey,'utf-8').decode('unicode_escape').encode('latin1')
bob_privkey = readFile('protocol-1-part-2-bob/bob_privkey.txt')
bob_privkey = bytes(bob_privkey,'utf-8').replace(b'\\n', b'\n')
decrypted_symkey = RSA_decrypt(private_key, alice_encryptedsymkey)
makeFile('protocol-1-part-2-bob','decrypted_symkey.txt', str(decrypted_symkey)[2:-1])

# 6. Alice encrypts her message using symkeyK
message = b"No pain no gain. No guts no glory. Cyberus is BEST.................................."  # this needs to be a multiple of 16 bytes for AES in this mode
symkeyK = readFile('protocol-1-part-2-alice/symkey.txt')
symkeyK = bytes(symkeyK,'utf-8').decode('unicode_escape').encode('latin1')
cipher_text = AES_encrypt(symkeyK, bytes(message))
makeFile('protocol-1-part-2-alice','ciphertext_message.txt', str(cipher_text)[2:-1])
print("Cipher Text: ", cipher_text)
# Alice sends the ciphertext_mesage.txt to Bob

# 7. Bob decrypts the message using K
cipher_text_from_alice = readFile('protocol-1-part-2-alice/ciphertext_message.txt')
cipher_text_from_alice = bytes(cipher_text_from_alice,'utf-8').decode('unicode_escape').encode('latin1')
decrypted_Symkey = readFile('protocol-1-part-2-bob/decrypted_symkey.txt')
decrypted_Symkey = bytes(decrypted_Symkey,'utf-8').decode('unicode_escape').encode('latin1')
decrypted_message = AES_decrypt(decrypted_Symkey, cipher_text_from_alice)

print("Message:", decrypted_message.decode())



Creating protocol-1-part-2-bob/bob_privkey.txt...

Creating protocol-1-part-2-bob/bob_pubkey.txt...

Creating protocol-1-part-2-alice/symkey.txt...

Creating protocol-1-part-2-alice/encryptedsymkey.txt...

Creating protocol-1-part-2-bob/decrypted_symkey.txt...

Creating protocol-1-part-2-alice/ciphertext_message.txt...
Cipher Text:  b'\xbf\xd9\xf6\xe1\x02\xf3\x01Q[\t}h\xbe\xd2\x00\xd0\x8dN\xb2\xb4\x9d\xe0\xe3\x04I\xe3V6BG\x13\xfb\xad\x14\xeaR\x1c\x7fD\t\xf21\xfcA\x92&\xd5\xf2\xd0\x1a\xc0\xc3C\x16fE\xa6\x14\xc4\xdb\xbbJE6\xef\xdf <\xdb\x07h\x9b\x81\xdd(\x89\xd3\xa6)\xf34U\x00\x1e\xd7\xbex]\xd8\r\x0fhQ\xa5w\xd8&@\x96\x86|y<\xccT3u\xc1\xc7\xe3\xff\xe5'
Message: No pain no gain. No guts no glory. Cyberus is BEST..................................


# Protocol 2:  The RSA Signature Protocol

In [46]:
def makeFile(dir, fileName, content):
    # Our safety check will prevent us from overwriting our old key files:
    isExist = os.path.exists(dir)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(dir)
    if os.path.exists('%s/%s' % (dir, fileName)):
        sys.exit('WARNING: The file %s/%s already exists! Use a different name or delete these files and re-run this program.' % (dir, fileName))

    print()
    print('Creating %s/%s...' % (dir, fileName))
    fo = open('%s/%s' % (dir, fileName), 'w')
    fo.write(content)
    fo.close()

def readFile(dirfileName):
    fo = open(dirfileName)
    content = fo.read()
    fo.close()
    return content

In [47]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA256
import os

# 1. Key generation, message signing, and sending message
def generate_RSA_key_pair():
    # Key generation
    key = RSA.generate(2048)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

def signature_computation(message,private_key):
    # Hashing the message
    hashed_message = SHA256.new(message.encode())

    # Signing the hashed message
    signer = PKCS1_v1_5.new(RSA.import_key(private_key))
    signature = signer.sign(hashed_message)
    
    return signature

# 2. Message and signature verification
def signature_verification(sent_message, sent_signature, public_key):
    # Hashing the received message
    hashed_sent_message = SHA256.new(sent_message.encode())

    # Verifying the signature
    verifier = PKCS1_v1_5.new(RSA.import_key(public_key))
    verified_status = verifier.verify(hashed_sent_message, sent_signature)

    return verified_status


In [48]:
file_path_list = ['protocol-2-bob/plaintext-message.txt','protocol-2-bob/signature.txt','protocol-2-bob/bob_privkey.txt','protocol-2-bob/bob_pubkey.txt']
for file_path in file_path_list:
    if os.path.exists(file_path):
        os.remove(file_path)

message = "No pain no gain. No guts no glory. Cyberus is BEST.................................."
makeFile('protocol-2-bob','plaintext-message.txt', message)

# Bob generates the private_key and public_key
private_key, public_key = generate_RSA_key_pair()
makeFile('protocol-2-bob','bob_privkey.txt', str(private_key)[2:-1])
makeFile('protocol-2-bob','bob_pubkey.txt', str(public_key)[2:-1])
# Bob will compute the message with private_key to get the signature
signature = signature_computation(message, private_key)
makeFile('protocol-2-bob','signature.txt', str(signature)[2:-1])
# Bob stores it into the signature.txt file and sends it with bob_pubkey.txt and plaintext-message.txt to Alice

# Alice receives the plaintext-message.txt, bob_pubkey.txt, signature.txt, and uses the signature_verification to verify if the plaintext-message is exactly the original one which is computed with private_key to generate signature
sent_message = readFile('protocol-2-bob/plaintext-message.txt')
sent_publickey = readFile('protocol-2-bob/bob_pubkey.txt')
sent_publickey = bytes(sent_publickey,'utf-8').replace(b'\\n', b'\n')
sent_signature = readFile('protocol-2-bob/signature.txt')
sent_signature = bytes(sent_signature,'utf-8').decode('unicode_escape').encode('latin1')
verified_status = signature_verification(sent_message, sent_signature, sent_publickey)

# Display results
print("Message:", sent_message)
print("Verified Status:", verified_status)


Creating protocol-2-bob/plaintext-message.txt...

Creating protocol-2-bob/bob_privkey.txt...

Creating protocol-2-bob/bob_pubkey.txt...

Creating protocol-2-bob/signature.txt...
Message: No pain no gain. No guts no glory. Cyberus is BEST..................................
Verified Status: True


# Protocol 3: The Diffie–Hellman Key Exchange protocol

In [49]:
def makeFile(dir, fileName, content):
    # Our safety check will prevent us from overwriting our old key files:
    isExist = os.path.exists(dir)
    if not isExist:
        # Create a new directory because it does not exist
        os.makedirs(dir)
    if os.path.exists('%s/%s' % (dir, fileName)):
        sys.exit('WARNING: The file %s/%s already exists! Use a different name or delete these files and re-run this program.' % (dir, fileName))

    print()
    print('Creating %s/%s...' % (dir, fileName))
    fo = open('%s/%s' % (dir, fileName), 'w')
    fo.write(content)
    fo.close()

def readFile(dirfileName):
    fo = open(dirfileName)
    content = fo.read()
    fo.close()
    return content

In [50]:
import os
from Crypto.Cipher import AES
from Crypto.Util.number import bytes_to_long, long_to_bytes
from Crypto.Util.Padding import pad, unpad

# Parameters for Diffie-Hellman
p = 31  # prime number
a = 11  # primitive root modulo p. 

def generate_keys():
    #Generate private and public keys.
    private_key = os.urandom(1)[0] % (p-1) + 1  # Ensure we're in {1,...,p-1}
    public_key = pow(a, private_key, p)
    return private_key, public_key

def compute_shared_secret(private_key, other_public_key):
    #Compute the shared secret key.
    return pow(other_public_key, private_key, p)

def aes_encrypt(key, plaintext):
    #Encrypt plaintext using AES with the given key.
    cipher = AES.new(long_to_bytes(key).rjust(16, b'\0'), AES.MODE_OFB, iv=b'CYBERUSIV2211200')
    return cipher.iv + cipher.encrypt(pad(plaintext.encode(), AES.block_size))

def aes_decrypt(key, ciphertext):
    #Decrypt ciphertext using AES with the given key.
    cipher = AES.new(long_to_bytes(key).rjust(16, b'\0'), AES.MODE_OFB, iv=ciphertext[:16])
    return unpad(cipher.decrypt(ciphertext[16:]), AES.block_size).decode()

In [51]:
file_path_list = ['protocol-3-bob/bob_shared_secretkey.txt','protocol-3-bob/bob_privkey.txt','protocol-3-bob/bob_pubkey.txt',
                 'protocol-3-alice/alice_shared_secretkey.txt','protocol-3-alice/alice_privkey.txt','protocol-3-alice/alice_pubkey.txt','protocol-3-alice/ciphertext-message.txt']
for file_path in file_path_list:
    if os.path.exists(file_path):
        os.remove(file_path)

# Alice generates her keys
alice_private_key, alice_public_key = generate_keys()
makeFile('protocol-3-alice','alice_privkey.txt',str(alice_private_key))
makeFile('protocol-3-alice','alice_pubkey.txt',str(alice_public_key))

# Bob generates his keys
bob_private_key, bob_public_key = generate_keys()
makeFile('protocol-3-bob','bob_privkey.txt',str(bob_private_key))
makeFile('protocol-3-bob','bob_pubkey.txt',str(bob_public_key))

# Bob compute the shared secret key
alice_sent_pub_key = int(readFile('protocol-3-alice/alice_pubkey.txt'))
bob_shared_secretkey = compute_shared_secret(bob_private_key, alice_sent_pub_key)
makeFile('protocol-3-bob','bob_shared_secretkey.txt',str(bob_shared_secretkey))

# Alice compute the shared secret key
bob_sent_pub_key = int(readFile('protocol-3-bob/bob_pubkey.txt'))
alice_shared_secretkey = compute_shared_secret(alice_private_key, bob_sent_pub_key)
makeFile('protocol-3-alice','alice_shared_secretkey.txt',str(alice_shared_secretkey))

# Alice sends an encrypted message to Bob
message = "No pain no gain. No guts no glory. Cyberus is BEST.................................."
cipher_text = aes_encrypt(alice_shared_secretkey, message)
print()
print("Cipher Text:", cipher_text)
makeFile('protocol-3-alice','ciphertext-message.txt',str(cipher_text)[2:-1])

# Bob decrypts the message
sent_cipher_text = readFile('protocol-3-alice/ciphertext-message.txt')
sent_cipher_text = bytes(sent_cipher_text,'utf-8').decode('unicode_escape').encode('latin1')
decrypted_message = aes_decrypt(bob_shared_secretkey, sent_cipher_text)
print("Decrypted Message:", decrypted_message)


Creating protocol-3-alice/alice_privkey.txt...

Creating protocol-3-alice/alice_pubkey.txt...

Creating protocol-3-bob/bob_privkey.txt...

Creating protocol-3-bob/bob_pubkey.txt...

Creating protocol-3-bob/bob_shared_secretkey.txt...

Creating protocol-3-alice/alice_shared_secretkey.txt...

Cipher Text: b'CYBERUSIV2211200\xba_\x1cW\t\xa7Y\x10\x8e\x9dZ\xe5w\xfdrWM\xaa\xd8"\x93\x8d\x19\x7f\xd8\xac\x9b\xa4\x04\xda6\x96\xc8+\xa0\xc8\xb4p\xedX\x0c\xf9\x9b^\xa7\x02\xce\xa1B\xe1Zm\xcdjI|dM\x920\x98\x84O\xf3\x05\x10/\xe1\x01\xd9\x89\x92q[F\xb9\x19\x89\xf0\xe3\x8c\xeb\x01\xe0-\xa4T\xf7aA\x04\x0f\xc61H\x14'

Creating protocol-3-alice/ciphertext-message.txt...
Decrypted Message: No pain no gain. No guts no glory. Cyberus is BEST..................................
